In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//h3[@class="title01"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                src = 'http://www.fenderliner.com' + html.xpath('//div[@class="b-img"]/img/@src')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_ul = html.xpath('//ul[@class="show-info-list"]/li')

                # = = = = = = = = = = = = = = =
                
                list_h4 = html.xpath('//h4[@class="title03"]')
                for h4 in list_h4:
                    if h4.xpath('./text()')[0].strip() == 'Description':
                        list_li = h4.xpath('./following-sibling::div[@class="edit"]')[0].xpath('./ul[@class="about-list"]/li/text()')
                
                                    
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])
                
                for ul in list_ul:
                    if len(ul.xpath('./@style')) != 0:
                        continue
                    
                    name = ul.xpath('./span/text()')[0].strip()[:-1].strip()
                    if name == 'Certifed by':
                        df_temp.loc[0, name.upper()] = '' if len(ul.xpath('./img')) == 0 else ';'.join([_.strip() for _ in ul.xpath('./img/@alt')])
                    else:
                        df_temp.loc[0, name.upper()] = '' if len(ul.xpath('./text()')) == 0 else ul.xpath('./text()')[0].strip()
                
                for li in list_li:
                    df_temp.loc[0, li.split(':')[0].strip().lower()] = ' : '.join(_.strip() for _ in li.split(':')[1:])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =

        print(f'''[{crawler_status}] - {input_.loc[a, 'Url']}\n[尝试次数：{b+1}] - [剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：58

[ok] - http://www.fenderliner.com/products-show.asp?seq=955
[尝试次数：1] - [剩余数量：38] - [当前时间：17:06:10]

[ok] - http://www.fenderliner.com/products-show.asp?seq=992
[尝试次数：1] - [剩余数量：37] - [当前时间：17:06:10]

[ok] - http://www.fenderliner.com/products-show.asp?seq=956
[尝试次数：1] - [剩余数量：36] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=960
[尝试次数：1] - [剩余数量：35] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=962
[尝试次数：1] - [剩余数量：34] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=986
[尝试次数：1] - [剩余数量：33] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=964
[尝试次数：1] - [剩余数量：32] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=957
[尝试次数：2] - [剩余数量：31] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=996
[尝试次数：1] - [剩余数量：30] - [当前时间：17:06:12]

[ok] - http://www.fenderliner.com/products-show.asp?seq=988
[尝试次数：1] - [剩余数量：29] - [当前时间：17:

[ok] - 351773414211
[尝试次数：1] - [剩余数量：1377] - [当前时间：19:52:53]

[ok] - 372320500208
[尝试次数：1] - [剩余数量：1376] - [当前时间：19:52:53]

[ok] - 372297027378
[尝试次数：1] - [剩余数量：1375] - [当前时间：19:52:53]

[ok] - 373977015239
[尝试次数：1] - [剩余数量：1374] - [当前时间：19:52:54]

[ok] - 233088271161
[尝试次数：2] - [剩余数量：1373] - [当前时间：19:52:54]

[ok] - 352756984299
[尝试次数：1] - [剩余数量：1372] - [当前时间：19:52:54]

[ok] - 371671964089
[尝试次数：3] - [剩余数量：1371] - [当前时间：19:52:54]

[ok] - 351641295628
[尝试次数：1] - [剩余数量：1370] - [当前时间：19:52:55]

[ok] - 232785731035
[尝试次数：1] - [剩余数量：1369] - [当前时间：19:52:55]

[ok] - 351887713417
[尝试次数：1] - [剩余数量：1368] - [当前时间：19:52:55]

[ok] - 374168941528
[尝试次数：1] - [剩余数量：1367] - [当前时间：19:52:55]

[ok] - 371701027054
[尝试次数：1] - [剩余数量：1366] - [当前时间：19:52:55]

[ok] - 374150827265
[尝试次数：1] - [剩余数量：1365] - [当前时间：19:52:56]

[ok] - 373651435852
[尝试次数：1] - [剩余数量：1364] - [当前时间：19:52:56]

[ok] - 372550249657
[尝试次数：6] - [剩余数量：1363] - [当前时间：19:52:56]

[ok] - 371008616889
[尝试次数：2] - [剩余数量：1362] - [当前时间：19:52:57]

[ok] - 3

[ok] - 353578634818
[尝试次数：1] - [剩余数量：1244] - [当前时间：19:53:33]

[ok] - 372544062116
[尝试次数：1] - [剩余数量：1243] - [当前时间：19:53:33]

[ok] - 233147980578
[尝试次数：2] - [剩余数量：1242] - [当前时间：19:53:33]

[ok] - 233114553742
[尝试次数：1] - [剩余数量：1241] - [当前时间：19:53:34]

[ok] - 354130612039
[尝试次数：1] - [剩余数量：1240] - [当前时间：19:53:35]

[ok] - 372261192290
[尝试次数：3] - [剩余数量：1239] - [当前时间：19:53:35]

[ok] - 371672101629
[尝试次数：1] - [剩余数量：1238] - [当前时间：19:53:35]

[ok] - 352565017616
[尝试次数：1] - [剩余数量：1237] - [当前时间：19:53:35]

[ok] - 352555064056
[尝试次数：4] - [剩余数量：1236] - [当前时间：19:53:35]

[ok] - 373960578080
[尝试次数：1] - [剩余数量：1235] - [当前时间：19:53:36]

[ok] - 354870237022
[尝试次数：1] - [剩余数量：1234] - [当前时间：19:53:36]

[ok] - 372602091431
[尝试次数：1] - [剩余数量：1233] - [当前时间：19:53:36]

[ok] - 355697411270
[尝试次数：1] - [剩余数量：1232] - [当前时间：19:53:37]

[ok] - 232780273664
[尝试次数：1] - [剩余数量：1231] - [当前时间：19:53:37]

[ok] - 352703127766
[尝试次数：1] - [剩余数量：1230] - [当前时间：19:53:37]

[ok] - 375398507937
[尝试次数：6] - [剩余数量：1229] - [当前时间：19:53:38]

[ok] - 3

[ok] - 353522027565
[尝试次数：1] - [剩余数量：1111] - [当前时间：19:54:14]

[ok] - 352762191241
[尝试次数：1] - [剩余数量：1110] - [当前时间：19:54:15]

[ok] - 234041084086
[尝试次数：1] - [剩余数量：1109] - [当前时间：19:54:15]

[ok] - 371497335475
[尝试次数：1] - [剩余数量：1108] - [当前时间：19:54:15]

[ok] - 231770749277
[尝试次数：1] - [剩余数量：1107] - [当前时间：19:54:15]

[ok] - 234093025373
[尝试次数：1] - [剩余数量：1106] - [当前时间：19:54:16]

[ok] - 234041067945
[尝试次数：1] - [剩余数量：1105] - [当前时间：19:54:16]

[ok] - 234198958820
[尝试次数：1] - [剩余数量：1104] - [当前时间：19:54:16]

[ok] - 352553646252
[尝试次数：1] - [剩余数量：1103] - [当前时间：19:54:17]

[ok] - 374307063948
[尝试次数：2] - [剩余数量：1102] - [当前时间：19:54:17]

[ok] - 353577013962
[尝试次数：1] - [剩余数量：1101] - [当前时间：19:54:17]

[ok] - 373652593160
[尝试次数：1] - [剩余数量：1100] - [当前时间：19:54:17]

[ok] - 232713115133
[尝试次数：1] - [剩余数量：1099] - [当前时间：19:54:18]

[ok] - 374298353242
[尝试次数：2] - [剩余数量：1098] - [当前时间：19:54:19]

[ok] - 231998070840
[尝试次数：5] - [剩余数量：1097] - [当前时间：19:54:19]

[ok] - 234757921371
[尝试次数：1] - [剩余数量：1096] - [当前时间：19:54:20]

[ok] - 2

[ok] - 355043561097
[尝试次数：1] - [剩余数量：978] - [当前时间：19:55:24]

[ok] - 233061631724
[尝试次数：2] - [剩余数量：977] - [当前时间：19:55:24]

[ok] - 375417494804
[尝试次数：3] - [剩余数量：976] - [当前时间：19:55:24]

[ok] - 352559970304
[尝试次数：3] - [剩余数量：975] - [当前时间：19:55:24]

[ok] - 372747140595
[尝试次数：2] - [剩余数量：974] - [当前时间：19:55:25]

[ok] - 355043561510
[尝试次数：3] - [剩余数量：973] - [当前时间：19:55:25]

[ok] - 371774023710
[尝试次数：1] - [剩余数量：972] - [当前时间：19:55:26]

[ok] - 234093035079
[尝试次数：1] - [剩余数量：971] - [当前时间：19:55:26]

[ok] - 232028471546
[尝试次数：3] - [剩余数量：970] - [当前时间：19:55:26]

[ok] - 354113198887
[尝试次数：3] - [剩余数量：969] - [当前时间：19:55:27]

[ok] - 233289538846
[尝试次数：2] - [剩余数量：968] - [当前时间：19:55:27]

[ok] - 371826383436
[尝试次数：2] - [剩余数量：967] - [当前时间：19:55:27]

[ok] - 351641289147
[尝试次数：16] - [剩余数量：966] - [当前时间：19:55:28]

[ok] - 355670099420
[尝试次数：1] - [剩余数量：965] - [当前时间：19:55:29]

[ok] - 234667446137
[尝试次数：3] - [剩余数量：964] - [当前时间：19:55:29]

[ok] - 372318954844
[尝试次数：1] - [剩余数量：963] - [当前时间：19:55:29]

[ok] - 373544371327
[尝试

...

[ok] - 234862820718
[尝试次数：5] - [剩余数量：574] - [当前时间：19:58:15]

[ok] - 373652551687
[尝试次数：1] - [剩余数量：573] - [当前时间：19:58:16]

[ok] - 234728265270
[尝试次数：3] - [剩余数量：572] - [当前时间：19:58:16]

[ok] - 355498489223
[尝试次数：5] - [剩余数量：571] - [当前时间：19:58:17]

[ok] - 371690047386
[尝试次数：7] - [剩余数量：570] - [当前时间：19:58:17]

[ok] - 351773476936
[尝试次数：3] - [剩余数量：569] - [当前时间：19:58:17]

[ok] - 234517438239
[尝试次数：1] - [剩余数量：568] - [当前时间：19:58:18]

[ok] - 352380889938
[尝试次数：2] - [剩余数量：567] - [当前时间：19:58:19]

[ok] - 374298352732
[尝试次数：1] - [剩余数量：566] - [当前时间：19:58:19]

[ok] - 352658703659
[尝试次数：1] - [剩余数量：565] - [当前时间：19:58:20]

[ok] - 371497329434
[尝试次数：1] - [剩余数量：564] - [当前时间：19:58:21]

[ok] - 353577013984
[尝试次数：1] - [剩余数量：563] - [当前时间：19:58:21]

[ok] - 354130612216
[尝试次数：1] - [剩余数量：562] - [当前时间：19:58:21]

[ok] - 374150827359
[尝试次数：1] - [剩余数量：561] - [当前时间：19:58:22]

[ok] - 232780431051
[尝试次数：15] - [剩余数量：560] - [当前时间：19:58:22]

[ok] - 354413355269
[尝试次数：1] - [剩余数量：559] - [当前时间：19:58:23]

[ok] - 373652545730
[尝试

[ok] - 234862820377
[尝试次数：2] - [剩余数量：439] - [当前时间：19:59:11]

[ok] - 234612990266
[尝试次数：2] - [剩余数量：438] - [当前时间：19:59:11]

[ok] - 373960573962
[尝试次数：2] - [剩余数量：437] - [当前时间：19:59:11]

[ok] - 375433675374
[尝试次数：1] - [剩余数量：436] - [当前时间：19:59:12]

[ok] - 352364744736
[尝试次数：5] - [剩余数量：435] - [当前时间：19:59:12]

[ok] - 355263340709
[尝试次数：1] - [剩余数量：434] - [当前时间：19:59:12]

[ok] - 235391005568
[尝试次数：12] - [剩余数量：433] - [当前时间：19:59:13]

[ok] - 372321482834
[尝试次数：2] - [剩余数量：432] - [当前时间：19:59:14]

[ok] - 375311404896
[尝试次数：9] - [剩余数量：431] - [当前时间：19:59:15]

[ok] - 234624888222
[尝试次数：1] - [剩余数量：430] - [当前时间：19:59:15]

[ok] - 374464811900
[尝试次数：1] - [剩余数量：429] - [当前时间：19:59:15]

[ok] - 235394758268
[尝试次数：1] - [剩余数量：428] - [当前时间：19:59:17]

[ok] - 355251799507
[尝试次数：1] - [剩余数量：427] - [当前时间：19:59:18]

[ok] - 355387646198
[尝试次数：4] - [剩余数量：426] - [当前时间：19:59:19]

[ok] - 374367403971
[尝试次数：1] - [剩余数量：425] - [当前时间：19:59:19]

[ok] - 353414826499
[尝试次数：3] - [剩余数量：424] - [当前时间：19:59:20]

[ok] - 233172117938
[尝试

[ok] - 233071966546
[尝试次数：1] - [剩余数量：305] - [当前时间：20:00:36]

[ok] - 355251800675
[尝试次数：1] - [剩余数量：304] - [当前时间：20:00:36]

[ok] - 372563377152
[尝试次数：14] - [剩余数量：303] - [当前时间：20:00:36]

[ok] - 234795813050
[尝试次数：1] - [剩余数量：302] - [当前时间：20:00:37]

[ok] - 354323706412
[尝试次数：15] - [剩余数量：301] - [当前时间：20:00:37]

[ok] - 354694202802
[尝试次数：1] - [剩余数量：300] - [当前时间：20:00:37]

[ok] - 355223721612
[尝试次数：1] - [剩余数量：299] - [当前时间：20:00:38]

[ok] - 352565020143
[尝试次数：1] - [剩余数量：298] - [当前时间：20:00:39]

[ok] - 375265256011
[尝试次数：1] - [剩余数量：297] - [当前时间：20:00:39]

[ok] - 233465784052
[尝试次数：1] - [剩余数量：296] - [当前时间：20:00:40]

[ok] - 373603476120
[尝试次数：1] - [剩余数量：295] - [当前时间：20:00:40]

[ok] - 373604518892
[尝试次数：1] - [剩余数量：294] - [当前时间：20:00:40]

[ok] - 234957301405
[尝试次数：1] - [剩余数量：293] - [当前时间：20:00:40]

[ok] - 234927459432
[尝试次数：2] - [剩余数量：292] - [当前时间：20:00:41]

[ok] - 354774398415
[尝试次数：1] - [剩余数量：291] - [当前时间：20:00:41]

[ok] - 374464811951
[尝试次数：4] - [剩余数量：290] - [当前时间：20:00:41]

[ok] - 373652531586
[尝

[ok] - 354424611482
[尝试次数：1] - [剩余数量：168] - [当前时间：20:01:28]

[ok] - 372497372386
[尝试次数：1] - [剩余数量：167] - [当前时间：20:01:29]

[ok] - 372631171387
[尝试次数：1] - [剩余数量：166] - [当前时间：20:01:29]

[ok] - 235476885823
[尝试次数：1] - [剩余数量：165] - [当前时间：20:01:30]

[ok] - 372666375095
[尝试次数：1] - [剩余数量：164] - [当前时间：20:01:30]

[ok] - 354424612517
[尝试次数：1] - [剩余数量：163] - [当前时间：20:01:30]

[ok] - 375398502818
[尝试次数：1] - [剩余数量：162] - [当前时间：20:01:30]

[ok] - 355475275331
[尝试次数：2] - [剩余数量：161] - [当前时间：20:01:31]

[ok] - 354154251147
[尝试次数：1] - [剩余数量：160] - [当前时间：20:01:31]

[ok] - 354638862535
[尝试次数：1] - [剩余数量：159] - [当前时间：20:01:31]

[ok] - 375314299508
[尝试次数：1] - [剩余数量：158] - [当前时间：20:01:31]

[ok] - 234864414993
[尝试次数：1] - [剩余数量：157] - [当前时间：20:01:31]

[ok] - 352363180309
[尝试次数：1] - [剩余数量：156] - [当前时间：20:01:31]

[ok] - 233220719867
[尝试次数：1] - [剩余数量：155] - [当前时间：20:01:32]

[ok] - 372968479600
[尝试次数：1] - [剩余数量：154] - [当前时间：20:01:32]

[ok] - 233486063275
[尝试次数：1] - [剩余数量：153] - [当前时间：20:01:32]

[ok] - 354383319312
[尝试次

[ok] - 233994913978
[尝试次数：1] - [剩余数量：32] - [当前时间：20:02:14]

[ok] - 372563448253
[尝试次数：1] - [剩余数量：31] - [当前时间：20:02:14]

[ok] - 353960925723
[尝试次数：1] - [剩余数量：30] - [当前时间：20:02:15]

[ok] - 374838064073
[尝试次数：1] - [剩余数量：29] - [当前时间：20:02:15]

[ok] - 375314299516
[尝试次数：3] - [剩余数量：28] - [当前时间：20:02:16]

[ok] - 375311453367
[尝试次数：1] - [剩余数量：27] - [当前时间：20:02:16]

[ok] - 375334608656
[尝试次数：1] - [剩余数量：26] - [当前时间：20:02:17]

[ok] - 232797372035
[尝试次数：1] - [剩余数量：25] - [当前时间：20:02:17]

[ok] - 355581493940
[尝试次数：1] - [剩余数量：24] - [当前时间：20:02:17]

[ok] - 372497364535
[尝试次数：1] - [剩余数量：23] - [当前时间：20:02:18]

[ok] - 371605128058
[尝试次数：1] - [剩余数量：22] - [当前时间：20:02:18]

[ok] - 234927458690
[尝试次数：1] - [剩余数量：21] - [当前时间：20:02:18]

[ok] - 232785880060
[尝试次数：1] - [剩余数量：20] - [当前时间：20:02:18]

[ok] - 355719640816
[尝试次数：1] - [剩余数量：19] - [当前时间：20:02:19]

[ok] - 374177228711
[尝试次数：1] - [剩余数量：18] - [当前时间：20:02:19]

[ok] - 355371305668
[尝试次数：1] - [剩余数量：17] - [当前时间：20:02:20]

[ok] - 353576981579
[尝试次数：1] - [剩余数量：16]